In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv('../data/name_romanization.csv')
df['romanize1'] = df.romanize1.apply(lambda x: x.lower())
print('num of row:', len(df))
df = df.drop_duplicates()
print('drop duplicate:', len(df))
#df_name = pd.read_csv('name1000.csv')

num of row: 4054
drop duplicate: 3847


# make name list from original data

In [15]:
### make dict {thai: [roman1, roman2],...}
### one transcription may be read in several ways e.g. เอกรัฐ
names = {}
for g, p in zip(df.token, df.romanize1):
    names[g] = names.get(g, []) + [p]
print('num of distinct names:', len(names))

### exclude ambiguous names
names_not_amb = [(g,p[0]) for g,p in names.items() if len(p)==1]
print('not ambiguous name:', len(names_not_amb))

num of distinct names: 3766
not ambiguous name: 3766


In [16]:
### if use all entries
#df = df[['token',"romanize1"]]

### if use not ambiguous name
df = pd.DataFrame(names_not_amb, columns=['token', 'romanize1']) 

df

,token,romanize1
0,กุลสตรี,kunlasattri
1,กุสุมา,kusuma
2,กุสุมาลย์,kusuman
3,กุหลาบ,kulap
4,ก่อเกียรติ,kokiat
...,...,...
3761,ไลลา,laila
3762,ไวพจน์,waiphot
3763,ไอริณ,airin
3764,ไอลดา,ailada


In [17]:
train, test = train_test_split(df, test_size=0.1858)
print('train:', len(train))
print('test:', len(test))
print(len(test), "=", len(set(test.token))) # must equal

train: 3066
test: 700
700 = 700


In [18]:
train.token.to_csv('../data/train_x.txt', index=False, header=False)
train.romanize1.to_csv('../data/train_y.txt', index=False, header=False)
test.token.to_csv('../data/test_x.txt', index=False, header=False)
test.romanize1.to_csv('../data/test_y.txt', index=False, header=False)

# additional dict data

In [ ]:
df = pd.read_csv('../data/g2p_dict.csv')
df.g.to_csv('../data/dic_x.txt', index=False, header=None)
df.rtgs.to_csv('../data/dic_y.txt', index=False, header=None)